## Lista 5 - Aprendizagem de Máquina Probabilístico
- Aluno: Lucas Rodrigues Aragão - Graduação 538390

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import jax
import jax.numpy as jnp
from jax.scipy.linalg import cholesky


## Modelos

### Inferência com modelos de GP para regressão 
- Estimação 

    1. Inicializar hiperparâmetros $\theta = \big[  \sigma^2_f, w^2_1, \cdots, w^2_D, \sigma^2_y \big]^T$

    2. Repetir até convergir ou número de épocas

        1. Calcular a evidência do modelo $\log p(y|X, \theta)$. A evidência é calculada via Cholesky.
            $$K+ \sigma^2_y = LL^T, \alpha = L^{-1}y$$
            $$\mathcal{L} (\theta) = - \sum_{i}{\log L_{ii} - \frac{1}{2} \alpha^T \alpha - \frac{N}{2} \log (2 \pi)}$$
        2. Calcular os gradientes analíticos $\frac{\partial \log p(y|X, \theta)}{\partial \theta}$.
        3. Atualizar $\theta$ a partir dos gradientes.

    3. Retornar os hiperparâmetros

- Predição

1. Dado um novo padrão $x_\ast$ retornar a distribuição preditiva 

$$p(y| x_\ast, y, X, \hat{\theta}) = \mathcal{N}(y_\ast| \mu_\ast, \sigma^2_\ast + \sigma^2_y)$$

$$\mu_\ast = k_{f \ast}^T (K + \sigma^2_y I)^{-1} y$$

$$\sigma^2_\ast = k_{\ast \ast} - k_{f \ast}^T (K + \sigma^2_y I)^{-1} k_{f \ast}$$

Em que, $k_{f \ast} = [k(x_\ast, x_1), \cdots , k(x_\ast, x_N)]$ e $k_{\ast \ast} = k(x_\ast, x_\ast)$

Além disso, valores de inicialização comuns são, $\sigma^2_f = \mathbb{V}[y]$, $w^2_d = \frac{1}{\mathbb{V}[X_{:d}]}$ e $\sigma^2_y = 0.01 \sigma^2_f$, com o $\mathbb{V}$ sendo a variância.


In [ ]:
class RegGP:
    def __init__(self, sigma2f, w0, sigma2y):

        self.theta = jnp.array([sigma2f, w0, sigma2y])

    def RBF(self, theta, xi, xj):
        sigma2f = theta[0]
        diff = xi-xj
        w = theta[1:-1] 
        temp = jnp.sum(w * diff**2)
        return sigma2f * jnp.exp(-0.5 * temp)
        
    def apply_kernel(self, X, theta):
        return jax.vmap(lambda xi: jax.vmap(lambda xj: self.RBF(theta , xi, xj))(X))(X)


    def estimate(self, X_train, y_train ,epochs, lr):

        grad_evidence = jax.grad(self.evidence)
        theta = self.theta

        for epoch in range(epochs):
            #calcular a evidencia log P(y|X,theta), via cholesky            
            #calcular os gradientes 
            # atualizar os hiperparametros
            grads = grad_evidence(theta=theta,X=X_train, y= y_train)
            theta = theta + lr*grads
            
        self.theta = theta
        return theta
    
    def evidence(self, theta, X ,y):
        K = self.apply_kernel(theta, X)
        sigma2y = theta[-1]
        N = y.shape[0]
        L = cholesky(K + sigma2y * jnp.eye(N))
        alpha = jnp.linalg.solve(L.T, jnp.linalg.solve(L,y))
        evidencia = - jnp.sum(jnp.log(jnp.diagonal(L))) - 0.5 * alpha.T @ alpha - 0.5 * N * jnp.log(2 * jnp.pi)

        return evidencia

    def predict(self, X_ast):
        #TODO: Terminar predict
        pass